In [1]:
import pandas as pd
import warnings 

warnings.filterwarnings('ignore')

In [2]:
client_path = r'CAL00001 Raw ClientA-Run1-Client.csv'
lab_path = r'CAL00001 Raw ClientA-Run1-Lab.csv'

# Average Later

In [3]:
def get_one(path, beam):
    df = pd.read_csv(path, encoding='mac_roman')
    
    title_index = 0
    Backgrounds_num = 0
    Measurements_num = 0

    for i in range(len(df)):
        # get where the data start
        if 'DATA' in df.iloc[i].tolist()[0]:
            title_index = i + 1
            break

        # get the Background number
        elif 'Backgrounds' in df.iloc[i].tolist()[0]:
            Backgrounds_num = int(df.iloc[i].tolist()[2])

        # get the measurement number
        elif 'Measurements' in df.iloc[i].tolist()[0]:
            Measurements_num = int(df.iloc[i].tolist()[2])
        
    # store the title
    title = df.iloc[title_index].values.tolist()
    
    df_total = df[title_index + 1:]
    df_total.columns = title
    df_total.drop(['kV','BarCode', 'XraysOn', 'FilterReady','HVLReady', 'mA', 'HVLFilter(mm)', 'N', 'P(kPa)', 'Comment'], axis=1, inplace=True)

    # change the column type to numeric
    df_total = df_total.apply(pd.to_numeric, errors='ignore')
    
    # dataframe of the before xray data
    df_before = df_total[:Backgrounds_num]
    
    # dataframe of the beam data with open xray
    df_data = df_total[Backgrounds_num:-Backgrounds_num]
    df_data = df_data[df_data.Filter.isin([beam])]
    
    # dataframe of the after xray data
    df_after = df_total[-Backgrounds_num:]
    
    return df_before.groupby(['Filter']).mean(), df_data

In [4]:
client_before, client_data = get_one(client_path, 'NXJ70')
lab_before, lab_data = get_one(lab_path, 'NXJ70')

In [5]:
BgdMC1_Before = client_before['Current1(pA)'].values[0]
BgdIC1_Before = client_before['Current2(pA)'].values[0]

client_data['Current1(pA)'] = client_data['Current1(pA)'] - BgdMC1_Before
client_data['Current2(pA)'] = client_data['Current2(pA)'] - BgdIC1_Before
client_data['R'] = client_data['Current2(pA)'] / client_data['Current1(pA)']
client_mean = client_data.groupby(['Filter']).mean()

MC1 = client_mean['Current1(pA)'].values[0]
IC1 = client_mean['Current2(pA)'].values[0]
R1 = client_mean['R'].values[0]
TM1 = client_mean['T(MC)'].values[0]
TA1 = client_mean['T(Air)'].values[0]

In [6]:
BgdMC2_Before = lab_before['Current1(pA)'].values[0]
BgdIC2_Before = lab_before['Current2(pA)'].values[0]

lab_data['Current1(pA)'] = lab_data['Current1(pA)'] - BgdMC2_Before
lab_data['Current2(pA)'] = lab_data['Current2(pA)'] - BgdIC2_Before
lab_data['R'] = lab_data['Current2(pA)'] / lab_data['Current1(pA)']
lab_mean = lab_data.groupby(['Filter']).mean()

MC2 = lab_mean['Current1(pA)'].values[0]
IC2 = lab_mean['Current2(pA)'].values[0]
R2 = lab_mean['R'].values[0]
TM2 = lab_mean['T(MC)'].values[0]
TS2 = lab_mean['T(SC)'].values[0]
H2 = lab_mean['H(%)'].values[0]

In [7]:
# read constant and KK from constant excel file
constant = r'E:\Unimelb\2021 S2\Software Project\test\constant\constant.xlsx'
df_constant = pd.read_excel(constant, sheet_name='constant')
df_KK = pd.read_excel(constant, sheet_name='Beams', usecols=[0,9])

# get ma and WE
Ma = df_constant['ma'].values[0]
WE = df_constant['WE'].values[0]
kk = df_KK[df_KK.Filter.isin(['NXJ70'])]['Product'].values[0]

In [8]:
NK = R2 * WE * kk * ((273.15 + TS2) / (273.15 + TM2)) * (0.995766667 + 0.000045 * H2) / (Ma * R1 * (273.15 + TA1) / (273.15 + TM1))
NK = NK / 1000000

In [9]:
NK

49.367716198545274

In [10]:
print('Ma', Ma)
print('WE', WE)
print('kk', kk)

Ma 6.179781954096993e-06
WE 33.97
kk 1.0217669224126713


# Average First

In [11]:
client_path = r'CAL00001 Raw ClientA-Run1-Client.csv'
lab_path = r'CAL00001 Raw ClientA-Run1-Lab.csv'

In [12]:
def get_one(path, beam):
    df = pd.read_csv(path, encoding='mac_roman')
    
    title_index = 0
    Backgrounds_num = 0
    Measurements_num = 0

    for i in range(len(df)):
        # get where the data start
        if 'DATA' in df.iloc[i].tolist()[0]:
            title_index = i + 1
            break

        # get the Background number
        elif 'Backgrounds' in df.iloc[i].tolist()[0]:
            Backgrounds_num = int(df.iloc[i].tolist()[2])

        # get the measurement number
        elif 'Measurements' in df.iloc[i].tolist()[0]:
            Measurements_num = int(df.iloc[i].tolist()[2])
        
    # store the title
    title = df.iloc[title_index].values.tolist()
    
    df_total = df[title_index + 1:]
    df_total.columns = title
    df_total.drop(['kV','BarCode', 'XraysOn', 'FilterReady','HVLReady', 'mA', 'HVLFilter(mm)', 'N', 'P(kPa)', 'Comment'], axis=1, inplace=True)

    # change the column type to numeric
    df_total = df_total.apply(pd.to_numeric, errors='ignore')
    
    # dataframe of the before xray data
    df_before = df_total[:Backgrounds_num]
    
    # dataframe of the beam data with open xray
    df_data = df_total[Backgrounds_num:-Backgrounds_num]
    df_data = df_data[df_data.Filter.isin([beam])]
    
    # dataframe of the after xray data
    df_after = df_total[-Backgrounds_num:]
    
    return df_before.groupby(['Filter']).mean(), df_data.groupby(['Filter']).mean()

In [13]:
client_before, client_data = get_one(client_path, 'NXJ70')
lab_before, lab_data = get_one(lab_path, 'NXJ70')

In [14]:
# client calculation part
BgdMC1_Before = client_before['Current1(pA)'].values[0]
BgdIC1_Before = client_before['Current2(pA)'].values[0]

MC1 = (client_data['Current1(pA)'] - BgdMC1_Before).values[0]
IC1 = (client_data['Current2(pA)'] - BgdIC1_Before).values[0]
R1 = ((client_data['Current2(pA)'] - BgdIC1_Before) / (client_data['Current1(pA)'] - BgdMC1_Before)).values[0]
TM1 = client_data['T(MC)'].values[0]
TA1 = client_data['T(Air)'].values[0]

In [15]:
# lab calculation part
BgdMC2_Before = lab_before['Current1(pA)'].values[0]
BgdIC2_Before = lab_before['Current2(pA)'].values[0]

MC2 = (lab_data['Current1(pA)'] - BgdMC2_Before).values[0]
IC2 = (lab_data['Current2(pA)'] - BgdIC2_Before).values[0]
R2 = ((lab_data['Current2(pA)'] - BgdIC2_Before) / (lab_data['Current1(pA)'] - BgdMC2_Before)).values[0]
TM2 = lab_data['T(MC)'].values[0]
TS2 = lab_data['T(SC)'].values[0]
H2 = lab_data['H(%)'].values[0]

In [16]:
NK = R2 * WE * kk * ((273.15 + TS2) / (273.15 + TM2)) * (0.995766667 + 0.000045 * H2) / (Ma * R1 * (273.15 + TA1) / (273.15 + TM1))
NK = NK / 1000000

In [17]:
NK

49.3677158603548

In [18]:
print('Ma', Ma)
print('WE', WE)
print('kk', kk)

Ma 6.179781954096993e-06
WE 33.97
kk 1.0217669224126713
